In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the pivot table containing node data (e.g., sensor readings)
pivot_df = pd.read_csv('fifty_temperatures_motes.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)
pivot_df = pivot_df.head(500)  # Limit the dataset for simulation

# Parameters
penalty = 1  # Penalty for active action
gamma = 1  # Discount factor for future AoII
pdr = 0.3  # Probability of successful transmission
beta_1 = 0.3  # dEWMA parameter for state value
beta_2 = 0.1  # dEWMA parameter for rate of change
node_count_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]  # Number of nodes
M = 1  # Fixed number of nodes to poll
num_simulation_runs = 10  # Number of simulation runs

# Function to initialize the state of nodes
def initialize_states(num_nodes):
    state_node = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Node-side state
    state_sink = {f'mote{i}': np.array([20.0, 0.1]) for i in range(1, num_nodes + 1)}  # Sink-side state
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}  # Track last update times
    aoii_history = {i: [] for i in range(1, num_nodes + 1)}  # Store AoII values
    aoi_rr_history = {i: [] for i in range(1, num_nodes + 1)}  # Store AoI values for Round Robin
    aoi_whittle_history = {i: [] for i in range(1, num_nodes + 1)}  # Store AoI values for Whittle AoI
    return state_node, state_sink, last_update_times, aoii_history, aoi_rr_history, aoi_whittle_history

# Function to simulate transmission success based on probability
def transmission_success(prob):
    return np.random.rand() < prob

# Function to compute Round Robin AoI
def poll_nodes_rr(current_time, last_update_times, aoi_rr_history, num_nodes, transmission_probs):
    node_rr = (current_time - 1) % num_nodes  # Poll nodes in cyclic order
    if transmission_success(transmission_probs[node_rr]):  # Transmission success
        last_update_times[node_rr] = current_time  # Update last update time only on success
    aoi_rr = {node: current_time - last_update_times[node] for node in last_update_times}  # Update AoI for each node
    aoi_rr_history[node_rr].append(np.mean(list(aoi_rr.values())))  # Store average AoI for RR
    return last_update_times, aoi_rr_history

# Function to run the simulation for specific node counts
def run_simulation_varying_nodes(node_count_list, M, transmission_probs):
    avg_aoii_results, avg_aoi_rr_results, avg_aoi_whittle_results = [], [], []

    for num_nodes in node_count_list:
        aoii_all_runs, aoi_rr_all_runs, aoi_whittle_all_runs = [], [], []

        for _ in range(num_simulation_runs):
            # Initialize states and trackers
            state_node, state_sink, last_update_times, aoii_history, aoi_rr_history, aoi_whittle_history = initialize_states(num_nodes)

            for t in range(1, len(pivot_df) + 1):
                # Update the state for each node using dEWMA
                for mote in range(1, num_nodes + 1):
                    column_name = f'mote{mote}'
                    measured_value = pivot_df.loc[t - 1, column_name]  # Use t-1 for 0-based DataFrame access
                    last_state_value, last_rate_of_change = state_node[column_name]
                    state_node[column_name] = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t=1)

                # Poll nodes based on Round Robin (RR)
                last_update_times, aoi_rr_history = poll_nodes_rr(t, last_update_times, aoi_rr_history, num_nodes, transmission_probs)

                # Poll nodes based on Whittle Index AoI
                last_update_times, aoi_whittle_history = poll_nodes_whittle_aoi(t, last_update_times, aoi_whittle_history, num_nodes, transmission_probs)

                # Poll nodes based on Whittle Index AoII
                state_node, state_sink, last_update_times, aoii_history = poll_nodes_whittle_aoii(
                    pivot_df, t, state_node, state_sink, last_update_times, aoii_history, M, penalty, num_nodes, transmission_probs
                )

            # Compute the average AoI values for this run
            avg_aoii = np.mean([np.mean(history) for history in aoii_history.values() if history])
            avg_aoi_rr = np.mean([np.mean(history) for history in aoi_rr_history.values() if history])
            avg_aoi_whittle = np.mean([np.mean(history) for history in aoi_whittle_history.values() if history])

            # Store results of this run
            aoii_all_runs.append(avg_aoii)
            aoi_rr_all_runs.append(avg_aoi_rr)
            aoi_whittle_all_runs.append(avg_aoi_whittle)

        # Compute the average AoI values across all runs
        avg_aoii_results.append(np.mean(aoii_all_runs))
        avg_aoi_rr_results.append(np.mean(aoi_rr_all_runs))
        avg_aoi_whittle_results.append(np.mean(aoi_whittle_all_runs))

    return avg_aoii_results, avg_aoi_rr_results, avg_aoi_whittle_results

# Run the simulation and plot results
def run_and_plot_simulation():
    transmission_probs = np.full(max(node_count_list), pdr)  # Transmission probabilities

    # Run the simulation for the specified number of nodes and transmission probabilities
    avg_aoii_results, avg_aoi_rr_results, avg_aoi_whittle_results = run_simulation_varying_nodes(
        node_count_list, M=1, transmission_probs=transmission_probs
    )

    # Plot the results for all three scheduling policies
    plt.figure(figsize=(10, 6))
    plt.plot(node_count_list, avg_aoii_results, label='Whittle Index AoII', color='green', marker='o')
    plt.plot(node_count_list, avg_aoi_rr_results, label='Round Robin AoI', color='blue', marker='o')
    plt.plot(node_count_list, avg_aoi_whittle_results, label='Whittle Index AoI', color='red', marker='o')
    plt.xlabel('Total Number of Nodes', fontsize=14)
    plt.ylabel('Average AoI / AoII', fontsize=14)
    plt.title('Comparison of Whittle Index (AoII) vs Round Robin (AoI) vs Whittle AoI', fontsize=16)
    plt.legend(fontsize=12)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Run and plot the simulation
run_and_plot_simulation()
